In [5]:
import xarray as xr
import numpy  as np
# import seaborn as sns
import pandas as pd
import matplotlib.pyplot as plt
import skill_metrics as sm
from taylor_preprocess import taylor_count

# 泰勒图绘制

继续使用skillmetics包进行

2021.08.25补充

比较早期的使用skillmetics尝试的例子，对形状的控制还不是很好；需要得到每个区域的AM，JJA时段的空间pattern，并计算相关的SCC，RMSD等，绘制泰勒图

效果较差，后续对每个区域计算了年际时间序列来进行绘制，并且对形状进行了控制，再脚本 pre_ts_taylor_partial.ipynb 之中


In [6]:
# get partial mask_res collect into a dict
partial_list = ["NE", "NC", "YZ", "SW"]
path_in = "/raid52/yycheng/MPAS/REFERENCE/TEMP_DATA/partial_pre"
file_ref = "mask_sel_CN05.1_Pre_1961_2018_daily_025x025.nc"
file_vr = "mask_vr_pre_98-17.nc"
file_rcm = "mask_rcm_pre_98-17.nc"
partial_res = {} # partial mask_res
for i in partial_list:
    path_in_partial = path_in + "/" + i + "/mask_res" # refine path
    print(path_in_partial)
    partial_res[i] = taylor_count_space(path_in_partial, file_ref, "pre", file_vr, "precip_MPAS", file_rcm, "precip_MPAS")

/raid52/yycheng/MPAS/REFERENCE/TEMP_DATA/partial_pre/NE/mask_res
/raid52/yycheng/MPAS/REFERENCE/TEMP_DATA/partial_pre/NC/mask_res
/raid52/yycheng/MPAS/REFERENCE/TEMP_DATA/partial_pre/YZ/mask_res
/raid52/yycheng/MPAS/REFERENCE/TEMP_DATA/partial_pre/SW/mask_res


In [11]:
partial_res['NE']['vr']

{'sdev': array([1.        , 2.19684143, 1.78507875, 1.67252652, 2.05962672,
        1.31082304, 1.75299111]),
 'crmsd': array([0.        , 1.35948568, 0.97478721, 1.03981962, 1.55980098,
        0.76110332, 1.03033098]),
 'ccoef': array([1.        , 0.90537055, 0.90648549, 0.81198119, 0.68193986,
        0.81589151, 0.85893075])}

In [ ]:
# >> taylor_diagram
# 不是多个模型对同一时段的模拟，绘制taylor图只是说明下相关性
# 逐年的固定某个月份 下面的 rmsd_select_year 在 months 中的范围大致是相同的
# label = ['Non-Dimensional Observation', 'M1']
first_plot = 'off'
color_plot = ["m","y","c","g"]
partial_list = ["NE", "NC", "YZ", "SW"]
plt.style.use(['science', 'no-latex']) # 采用 安装的scinence绘图风格，matplotlib风格拓展
# fig,ax = plt.subplots(4,1,figsize=(16,12))
fig = plt.figure(figsize=(7, 7), dpi=80)
# ax1=fig.add_subplot(2,2,1)
# ax2=fig.add_subplot(2,2,3)
ind = 0
for partial_ind in partial_list:
    model_plot = partial_res[partial_ind]
    label1 = {'VR': 'r', 'RCM': 'b'}
    label2 = ["obs","4","5","6","7","8","ALL"]
    # label2 = 
    # label2 = {"obs":"b","4":"r","5":"g","6":"k","7":"r","8":"r","ALL":"r"}
    lsCOR = np.concatenate((np.arange(0.4,1.0,0.2), [0.7,0.9, 0.95, 0.99, 1]))

    sm.taylor_diagram(model_plot['vr']['sdev'],model_plot['vr']['crmsd'],model_plot['vr']['ccoef'], 
    markerLabel = label2, 
    numberPanels = 1,
    # markerLabel = , 
    # markerlegend = "on",
    tickRMS = np.arange(0,2.2,0.2), tickRMSangle = 60, 
    colRMS = 'k', styleRMS = ':', widthRMS = 1.0,
    tickCOR = lsCOR,
    showlabelsRMS = "on",
    titleRMS = "off",
    colCOR = "k", 
    widthCOR = 1.0,
    tickSTD = [0,1,1.25,1.5,2.,2.5], axismax = 2.5, 
    showlabelsSTD = 'on',
    rmslabelformat = ':.1f',
    styleOBS = '-', 
    colOBS = 'k',
    markerobs = 'o', 
    markerColor = color_plot[ind],
    markerLabelColor = 'r',
    markerSize = 8, 
    alpha = 0., 
    overlay = first_plot,
    titleOBS = 'observation')

    sm.taylor_diagram(model_plot['rcm']['sdev'],model_plot['rcm']['crmsd'],model_plot['rcm']['ccoef'], markerLabel =label2,
    markerColor = color_plot[ind],
    # markerlegend = "on",
    markerLabelColor = 'b', 
    titleOBS = 'observation',
    markerSize = 8 ,
    alpha = 1., 
    overlay='on')

    # ax2 = sm.taylor_diagram(model_plot['rcm']['sdev'],model_plot['rcm']['crmsd'],model_plot['rcm']['ccoef'], markerLabel =label2,
    # markerColor = color_plot[ind],
    # # markerlegend = "on",
    # markerLabelColor = 'b', 
    # titleOBS = 'observation',
    # markerSize = 8 ,
    # alpha = 1., 
    # overlay='off')


    first_plot = 'on'
    ind = ind + 1


# plt.title("title")
# color_plot = ["m","y","w","g"]
# partial_list = ["NE", "NC", "YZ", "SW"]
import matplotlib.patches as mpatches
patch = []
patch.append(mpatches.Patch(color='m', label='NE'))
patch.append(mpatches.Patch(color='y', label='NC'))
patch.append(mpatches.Patch(color='c', label='YZ'))
patch.append(mpatches.Patch(color='g', label='SW'))
plt.legend(handles= patch, loc=[0.85,0.85])

# legend_elements = [color='b', lw=4, label='Line'),
# Line2D([0], [0], marker='o', color='w', label='Scatter',
#                           markerfacecolor='g', markersize=15),
#                    Patch(facecolor='orange', edgecolor='r',
#                          label='Color Patch')]
# plt.legend(label1, loc='upper right')

# pic_output_path = "/m2data2/yycheng/playground/MPAS_case_study/case_template/92-25km_vr_rcm_postprocess/visualization/check_obs/check_cn051/pre/taylor/output_pic/"
plt.savefig("./output_pic/4regions",dpi=300, facecolor='white')
# del(ax)

每个区域单独绘图，绘制子图不太好办，尝试着先把每个区域的图单独弄出来

In [ ]:
# >> taylor_diagram
# 不是多个模型对同一时段的模拟，绘制taylor图只是说明下相关性
# 逐年的固定某个月份 下面的 rmsd_select_year 在 months 中的范围大致是相同的
# label = ['Non-Dimensional Observation', 'M1']
first_plot = 'off'
# color_plot = ["m","y","c","g"]
color_plot = ["r","b"]
plt.style.use(['science', 'no-latex']) # 采用 安装的scinence绘图风格，matplotlib风格拓展
# fig,ax = plt.subplots(4,1,figsize=(16,12))
# ax1=fig.add_subplot(2,2,1)
# ax2=fig.add_subplot(2,2,3)
ind = 0
partial_list = ["NE", "NC", "YZ", "SW"]
for partial_ind in partial_list:
    fig = plt.figure(figsize=(5, 5), dpi=80)
    model_plot = partial_res[partial_ind]
    label2 = ["obs","4","5","6","7","8","ALL"]
    # label2 = 
    # label2 = {"obs":"b","4":"r","5":"g","6":"k","7":"r","8":"r","ALL":"r"}
    lsCOR = np.concatenate((np.arange(0.4,1.0,0.2), [0.7,0.9, 0.95, 0.99, 1]))

    sm.taylor_diagram(model_plot['vr']['sdev'],model_plot['vr']['crmsd'],model_plot['vr']['ccoef'], 
    markerLabel = label2, 
    numberPanels = 1,
    # markerLabel = , 
    # markerlegend = "on",
    tickRMS = np.arange(0,2.,0.2), tickRMSangle = 60, 
    colRMS = 'k', styleRMS = ':', widthRMS = 1.0,
    tickCOR = lsCOR,
    showlabelsRMS = "on",
    titleRMS = "off",
    colCOR = "k", 
    widthCOR = 1.0,
    tickSTD = [0,1,1.25,1.5,2.,2.5], axismax = 2.5, 
    showlabelsSTD = 'on',
    rmslabelformat = ':.1f',
    styleOBS = '-', 
    colOBS = 'k',
    markerobs = 'o', 
    markerColor = color_plot[ind],
    markerLabelColor = 'r',
    markerSize = 8, 
    alpha = 0., 
    overlay = first_plot,
    titleOBS = 'observation')

    sm.taylor_diagram(model_plot['rcm']['sdev'],model_plot['rcm']['crmsd'],model_plot['rcm']['ccoef'], markerLabel =label2,
    markerColor = color_plot[ind + 1],
    # markerlegend = "on",
    markerLabelColor = 'b', 
    titleOBS = 'observation',
    markerSize = 8 ,
    alpha = 1., 
    overlay='on')

    # ax2 = sm.taylor_diagram(model_plot['rcm']['sdev'],model_plot['rcm']['crmsd'],model_plot['rcm']['ccoef'], markerLabel =label2,
    # markerColor = color_plot[ind],
    # # markerlegend = "on",
    # markerLabelColor = 'b', 
    # titleOBS = 'observation',
    # markerSize = 8 ,
    # alpha = 1., 
    # overlay='off')

    # ind = ind + 1


    plt.title(partial_ind)
    # color_plot = ["m","y","w","g"]
    # partial_list = ["NE", "NC", "YZ", "SW"]
    import matplotlib.patches as mpatches
    patch = []
    patch.append(mpatches.Patch(color='r', label='VR'))
    patch.append(mpatches.Patch(color='b', label='RCM'))
    # patch.append(mpatches.Patch(color='m', label='NE'))
    # patch.append(mpatches.Patch(color='y', label='NC'))
    # patch.append(mpatches.Patch(color='c', label='YZ'))
    # patch.append(mpatches.Patch(color='g', label='SW'))
    # plt.legend(handles= patch, loc=[0.85,0.85])

    # legend_elements = [color='b', lw=4, label='Line'),
    # Line2D([0], [0], marker='o', color='w', label='Scatter',
    #                           markerfacecolor='g', markersize=15),
    #                    Patch(facecolor='orange', edgecolor='r',
    #                          label='Color Patch')]
    # plt.legend(label1, loc='upper right')

    # pic_output_path = "/m2data2/yycheng/playground/MPAS_case_study/case_template/92-25km_vr_rcm_postprocess/visualization/check_obs/check_cn051/pre/taylor/output_pic/"
    plt.savefig("./output_pic/taylor_" + partial_ind,dpi=300, facecolor = 'white')
    # del(ax)